In [1]:
from queue import PriorityQueue

goal_state = [[1, 2, 3], [8, 4, 0], [7, 6, 5]]
initial_state = [[2, 3, 0], [1, 8, 4], [7, 6, 5]]

def h_misplaced(state):
    count = 0
    for i in range(3):
        for j in range(3):
            if state[i][j] != 0 and state[i][j] != goal_state[i][j]:
                count += 1
    return count

def get_blank_pos(state):
    for i in range(3):
        for j in range(3):
            if state[i][j] == 0:
                return i, j

def generate_moves(state):
    i, j = get_blank_pos(state)
    moves = []
    directions = [(0,1),(1,0),(0,-1),(-1,0)]
    for di, dj in directions:
        ni, nj = i + di, j + dj
        if 0 <= ni < 3 and 0 <= nj < 3:
            new_state = [row[:] for row in state]
            new_state[i][j], new_state[ni][nj] = new_state[ni][nj], new_state[i][j]
            moves.append(new_state)
    return moves

def best_first_search(start):
    visited = set()
    pq = PriorityQueue()
    pq.put((h_misplaced(start), start, 0))
    
    while not pq.empty():
        h, state, moves = pq.get()
        print(f"Step {moves}:")
        for row in state:
            print(row)
        print("------")
        if state == goal_state:
            print(f"Goal reached in {moves} moves.")
            return
        visited.add(str(state))
        for next_state in generate_moves(state):
            if str(next_state) not in visited:
                pq.put((h_misplaced(next_state), next_state, moves+1))

best_first_search(initial_state)


Step 0:
[2, 3, 0]
[1, 8, 4]
[7, 6, 5]
------
Step 1:
[2, 0, 3]
[1, 8, 4]
[7, 6, 5]
------
Step 2:
[0, 2, 3]
[1, 8, 4]
[7, 6, 5]
------
Step 3:
[1, 2, 3]
[0, 8, 4]
[7, 6, 5]
------
Step 4:
[1, 2, 3]
[8, 0, 4]
[7, 6, 5]
------
Step 5:
[1, 2, 3]
[8, 4, 0]
[7, 6, 5]
------
Goal reached in 5 moves.


In [3]:
initial = [[2, 8, 3], [1, 5, 4], [7, 6, 0]]
goal = [[1, 2, 3], [8, 4, 0], [7, 6, 5]]

def hill_climbing(start):
    current = start
    while True:
        print("Current State:")
        for row in current:
            print(row)
        print("H(n):", h_misplaced(current))
        moves = generate_moves(current)
        next_move = min(moves, key=h_misplaced)
        if h_misplaced(next_move) >= h_misplaced(current):
            print("Local minimum reached.")
            break
        current = next_move
        if current == goal:
            print("Goal reached.")
            break

hill_climbing(initial)


Current State:
[2, 8, 3]
[1, 5, 4]
[7, 6, 0]
H(n): 5
Local minimum reached.


In [4]:
def h_correct(state):
    count = 0
    for i in range(3):
        for j in range(3):
            if state[i][j] == goal_state[i][j]:
                count += 1
    return 9 - count  # the fewer misplaced, the better

def a_star_search(start):
    visited = set()
    pq = PriorityQueue()
    pq.put((h_correct(start), 0, start))
    
    while not pq.empty():
        f, g, state = pq.get()
        print(f"g={g}, h={f - g}, f={f}")
        for row in state:
            print(row)
        print("------")
        if state == goal_state:
            print(f"Goal reached in {g} moves.")
            return
        visited.add(str(state))
        for next_state in generate_moves(state):
            if str(next_state) not in visited:
                h = h_correct(next_state)
                pq.put((g + 1 + h, g + 1, next_state))

a_star_search(initial_state)


g=0, h=6, f=6
[2, 3, 0]
[1, 8, 4]
[7, 6, 5]
------
g=1, h=5, f=6
[2, 0, 3]
[1, 8, 4]
[7, 6, 5]
------
g=1, h=5, f=6
[2, 3, 4]
[1, 8, 0]
[7, 6, 5]
------
g=2, h=4, f=6
[0, 2, 3]
[1, 8, 4]
[7, 6, 5]
------
g=3, h=3, f=6
[1, 2, 3]
[0, 8, 4]
[7, 6, 5]
------
g=4, h=2, f=6
[1, 2, 3]
[8, 0, 4]
[7, 6, 5]
------
g=5, h=0, f=5
[1, 2, 3]
[8, 4, 0]
[7, 6, 5]
------
Goal reached in 5 moves.


In [5]:
graph = {
    'S': [('A', 1), ('B', 5)],
    'A': [('C', 2), ('D', 10)],
    'B': [('E', 5), ('F', 5)],
    'C': [('G', 1)],
    'D': [('G', 1)],
    'E': [('G', 1)],
    'F': [('G', 1)],
    'G': []
}

heuristics = {
    'S': float('inf'),
    'A': float('inf'),
    'B': float('inf'),
    'C': 1,
    'D': 1,
    'E': 1,
    'F': 1,
    'G': 0
}

def ao_star(node):
    if not graph[node]:
        return heuristics[node]
    
    cost_list = []
    for child, edge_cost in graph[node]:
        cost = edge_cost + ao_star(child)
        cost_list.append((cost, child))
    
    min_cost, best_child = min(cost_list)
    heuristics[node] = min_cost
    return heuristics[node]

print("Minimum cost from S to goal G using AO* is:", ao_star('S'))


Minimum cost from S to goal G using AO* is: 4
